# Le jeu Palworld

## Nettoyage et Traitement des données

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import mariadb
from sqlalchemy import create_engine
import plotly.express as px
import plotly.graph_objects as go
import os
import csv
from pathlib import Path

## Chargement des données brutes

In [ ]:
# Définition du chemin vers les fichiers fournis
data_path = "C:/Users/ndiay/Desktop/lptf/projets/IA/DATA_ANALYSIS/JUPITER_NOTEBOOK/Analyse_pals/pals-analysis/data/data/"

# Vérification de l'existence du repertoire , if not utiliser le repertoire courrant
if not os.path.exists(data_path):
    data_path = "./"

# Affichage des fichiers dispo pour la vérif
files = [f for f in os.listdir(data_path) if f.endswith('.csv')] # ici on liste tous les fichiers ou dossiers + on itère chaque élément + Filtre pour garder que les fichier .csv
print("Fichiers dispo:")
for file in files: 
    print(f"-{file}")

# Chargement des données brutes & gestion d'erreurs grâce à pd.read_csv en définissant le chemin vers les fichiers.csv 
try:
    combat_attribute = pd.read_csv(f'{data_path}Palworld_Data--Palu combat attribute table.csv')
    refresh_area = pd.read_csv(f'{data_path}Palworld_Data--Palu refresh level.csv')
    ordinary_boss = pd.read_csv(f'{data_path}Palworld_Data-comparison of ordinary BOSS attributes.csv')
    tower_boss = pd.read_csv(f'{data_path}Palworld_Data-Tower BOSS attribute comparison.csv') 
    job_skill = pd.read_csv(f'{data_path}Palworld_Data-Palu Job Skills Table.csv')
    hidden_attribute = pd.read_csv(f'{data_path}Palworld_Data-hide pallu attributes.csv')
    print("\nDonnées brutes chargées")
except FileNotFoundError as e : 
  print(f"Erreur: {e}. Vérifier les noms des fichiers et le chemin.")

## Création de la fonction d'analyse exploratoires des données fournies

In [ ]:
def explore_dataframe(df, name):
    # Cette Fonction inclus un affichage optimisé
    
    # Sauvegarde et réglage des options d'affichage
    pd.set_option('display.max_columns', None)
    pd.set_option('display.max_colwidth', 30)
    pd.set_option('display.width', max(1200, df.shape[1] * 20))
    print(f"\n===== Exploration de {name} =====")
    
    # 1. Informations de base (similaire à df.info())
    print("\n--- Informations de base ---")
    print(f"Class: {type(df)}")
    # Index
    idx = df.index
    print(f"Index: {type(idx)} | {idx}")
    # Nombre total d'entrées et de colonnes
    print(f"Entries: {len(df)} | Columns: {df.shape[1]}")
    
    # Détail des colonnes et non-null counts
    structure = pd.DataFrame({
        'Column': df.columns,
        'Non-Null Count': df.notnull().sum().values,
        'Dtype': df.dtypes.values
    })
    display(structure)
    
    # Usage mémoire
    mem_usage = df.memory_usage(deep=True).sum()
    print(f"Memory usage: {mem_usage / 1024:.2f} KB")
    
    # 2. Dimensions
    print(f"\nDimensions: {df.shape[0]} lignes, {df.shape[1]} colonnes")
    
    # Aperçu des premières lignes
    print("\nAperçu des premières lignes:")
    display(df.head(3))
    
    # Types de données
    print("\nTypes de données:")
    display(df.dtypes)
    
    # Statistiques descriptives
    print("\nStatistiques descriptives:")
    display(df.describe(include='all'))
    
    # Valeurs manquantes
    print("\nValeurs manquantes:")
    missing = df.isnull().sum()
    missing_percent = (missing / len(df)) * 100
    missing_data = pd.concat([missing, missing_percent], axis=1, keys=['Total', 'Pourcentage'])
    display(missing_data[missing_data['Total'] > 0])
    if missing_data['Total'].sum() == 0:
        print("Aucune valeur manquante détectée")
    
    # Doublons
    duplicates = df.duplicated().sum()
    print(f"\nNombre de lignes dupliquées: {duplicates}")
    
    # Colonnes catégorielles
    print("\nAnalyse des colonnes catégorielles:")
    categorical_cols = df.select_dtypes(include=['object']).columns
    
    if len(categorical_cols) > 0:
        for col in categorical_cols:
            unique_values = df[col].unique()
            n_unique = len(unique_values)
            
            print(f"\n{col}:")
            print(f"  - {n_unique} valeurs uniques")
            
            if n_unique <= 10:
                print(f"  - Valeurs: {unique_values}")
            elif n_unique <= 20:
                value_counts = df[col].value_counts().head(10)
                print(f"  - Top 10 des valeurs les plus fréquentes:")
                for val, count in value_counts.items():
                    print(f"    • {val}: {count} occurrences")
            else:
                value_counts = df[col].value_counts()
                print(f"  - Top 5 des valeurs les plus fréquentes:")
                for val, count in value_counts.head(5).items():
                    print(f"    • {val}: {count} occurrences")
                print(f"  - Valeur la moins fréquente: {value_counts.tail(1).index[0]} ({value_counts.tail(1).values[0]} occurrence(s))")
    else:
        print("Aucune colonne catégorielle détectée")
    # Détection des valeurs aberrantes
    print("\nValeurs aberrantes (méthode IQR):")
    numeric_cols = df.select_dtypes(include=[np.number]).columns

 
        # Détection des valeurs aberrantes
    print("\nValeurs aberrantes (méthode IQR):")
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    
    for col in numeric_cols:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        outliers = df[(df[col] < Q1 - 1.5*IQR) | (df[col] > Q3 + 1.5*IQR)][col]
        
        print(f"{col}: {len(outliers)} valeurs aberrantes ({len(outliers)/len(df)*100:.1f}%)")
        
        # AFFICHAGE DES VALEURS ABERRANTES
        if len(outliers) > 0:
            if len(outliers) <= 10:
                print(f"  Valeurs: {sorted(outliers.tolist())}")
            else:
                print(f"  Premières 10: {sorted(outliers.tolist())[:10]}")
    # Rétablissement des options d'affichage
    pd.reset_option('display.max_columns')
    pd.reset_option('display.max_info_columns')
    pd.reset_option('display.max_colwidth')
    pd.reset_option('display.width')
    
    # Séparateur
    print("\n" + "="*70)

### Analyse exploratoire des données fournies

In [ ]:
explore_dataframe(job_skill, 'Job Skills')

In [ ]:
explore_dataframe(refresh_area, 'Refresh area')

In [ ]:
explore_dataframe(ordinary_boss, 'Ordinary Boss')

In [ ]:
explore_dataframe(tower_boss, 'Tower Boss')

In [ ]:
explore_dataframe(combat_attribute, 'Combat Attribut')

In [ ]:
explore_dataframe(hidden_attribute, 'Hidden Attribute')

## Création de la fonction d'analyse exploratoires des données propres

In [ ]:
# Définition du chemin vers les fichiers propres
data_path = "C:/Users/ndiay/Desktop/lptf/projets/IA/DATA_ANALYSIS/JUPITER_NOTEBOOK/Analyse_pals/pals-analysis/data/data2.0/"

# Vérification de l'existence du repertoire , if not utiliser le repertoire courrant
if not os.path.exists(data_path):
    data_path = "./"

# Affichage des fichiers dispo pour la vérif
files = [f for f in os.listdir(data_path) if f.endswith('.csv')] # ici on liste tous les fichiers ou dossiers + on itère chaque élément + Filtre pour garder que les fichier .csv
print("Fichiers dispo:")
for file in files: 
    print(f"-{file}")

# Chargement des données brutes & gestion d'erreurs grâce à pd.read_csv en définissant le chemin vers les fichiers.csv 
try:
    combat_attribute = pd.read_csv(f'{data_path}Clean_Data--Palu-combat-attribute.csv')
    refresh_area = pd.read_csv(f'{data_path}Clean_Data-Palu-refresh-level.csv')
    ordinary_boss = pd.read_csv(f'{data_path}Clean_Data-comparison-of-ordinary-BOSS-attributes.csv')
    tower_boss = pd.read_csv(f'{data_path}Clean_Data-Tower-BOSS-attribute-comparison.csv') 
    job_skill = pd.read_csv(f'{data_path}Clean_Data-Palu-Job-Skills.csv')
    hidden_attribute = pd.read_csv(f'{data_path}Clean_Data-hide-pallu-attributes.csv')
    print("\nDonnées brutes chargées")
except FileNotFoundError as e : 
  print(f"Erreur: {e}. Vérifier les noms des fichiers et le chemin.")

In [ ]:
explore_dataframe(job_skill, 'Job Skills')

In [ ]:
explore_dataframe(refresh_area, 'Refresh area')

In [ ]:
explore_dataframe(ordinary_boss, 'Ordinary Boss')

In [ ]:
explore_dataframe(tower_boss, 'Tower Boss')

In [ ]:
explore_dataframe(combat_attribute, 'Combat Attribut')

In [ ]:
explore_dataframe(hidden_attribute, 'Hidden Attribute')

### Création des tables, insertion des données et validations

In [ ]:
# Création table job_skills & Insertion des données dans les tables

# === Config base de données ===
config = {
    'user': 'root',
    'password': 'root',
    'host': 'localhost',
    'port': 3307,
    'database': 'palworld_database'
}

# === Connexion ===
try:
    conn = mariadb.connect(**config)
    cursor = conn.cursor()
    print("Connexion réussie à MariaDB !")
except mariadb.Error as e:
    print(f"Erreur de connexion : {e}")
    exit(1)

# === Création de la table job_skills ===
create_table_query = """
CREATE TABLE IF NOT EXISTS job_skills (
    id INT AUTO_INCREMENT PRIMARY KEY,
    `English name` VARCHAR(255),
    `Chinese name` VARCHAR(255),
    `Volume size` VARCHAR(50),
    `Food intake` INT,
    `night shift` BOOLEAN,
    `Total skills` INT,
    `Make a fire` BOOLEAN,
    `watering` BOOLEAN,
    `planting` BOOLEAN,
    `generate electricity` BOOLEAN,
    `manual` BOOLEAN,
    `collection` BOOLEAN,
    `logging` BOOLEAN,
    `Mining` BOOLEAN,
    `pharmaceutical` BOOLEAN,
    `cool down` BOOLEAN,
    `pasture` BOOLEAN,
    `carry` BOOLEAN,
    `Handling speed` BOOLEAN,
    `ranch items` VARCHAR(255),
    `pasture minimum output` VARCHAR(255),
    `The largest ranch` VARCHAR(255)
);
"""

try:
    cursor.execute(create_table_query)
    conn.commit()
    print("Table 'job_skills' prête.")
except mariadb.Error as e:
    print(f"Erreur création table : {e}")
    conn.close()
    exit(1)

# === Lecture CSV avec Pandas ===
csv_file_path = r'C:/Users/ndiay/Desktop/lptf/projets/IA/DATA_ANALYSIS/JUPITER_NOTEBOOK/Analyse_pals/pals-analysis/data/data2.0/Clean_Data-Palu-Job-Skills.csv'

try:
    df = pd.read_csv(csv_file_path)
    print(f"\nLe fichier CSV contient {df.shape[0]} lignes et {df.shape[1]} colonnes.")
except FileNotFoundError:
    print(f"Fichier non trouvé : {csv_file_path}")
    conn.close()
    exit(1)

# === Nombre de colonnes dans la table job_skills ===
cursor.execute("SELECT COUNT(*) FROM information_schema.columns WHERE table_name = 'job_skills' AND table_schema = 'palworld_database'")
column_count = cursor.fetchone()[0]
print(f"La table 'job_skills' contient maintenant {column_count} colonnes.")

# === Fonction de conversion ===
def convert(value, col_name):
    if value == '' or str(value).lower() == 'none':
        return None
    if col_name.lower() in ['night shift', 'make a fire', 'watering', 'planting', 'generate electricity', 'manual', 'collection', 'logging', 'mining', 'pharmaceutical', 'cool down', 'pasture', 'carry', 'handling speed']:
        return str(value).strip().lower() == 'true'
    value = value.replace(',', '.')
    try:
        if '.' in value:
            return float(value)
        return int(value)
    except ValueError:
        return value

# === Fonction de nettoyage pour les colonnes name ===
def clean_name(name):
    if name is None:
        return None
    # Convertir en minuscules, supprimer les underscores et les espaces
    return str(name).lower().replace('_', '').replace(' ', '')

# === Insertion des données dans job_skills ===
try:
    with open(csv_file_path, mode='r', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        headers = reader.fieldnames

        sql_headers = [h for h in headers]  # on conserve les noms initiaux
        placeholders = ', '.join(['%s'] * len(sql_headers))
        field_names = ', '.join(f'`{col}`' for col in sql_headers)
        insert_query = f"INSERT INTO job_skills ({field_names}) VALUES ({placeholders})"

        count = 0
        for row in reader:
            values = []
            for i, h in enumerate(headers):
                value = convert(row[h], h)
                # Nettoyer spécifiquement les colonnes "English name" et "Chinese name"
                if h.lower() in ['english name', 'chinese name']:
                    value = clean_name(value)
                values.append(value)
            try:
                cursor.execute(insert_query, values)
                count += 1
            except mariadb.Error as e:
                print(f"Erreur insertion ligne {count + 1}: {e} — valeurs: {values}")

        conn.commit()
        print(f"\n{count} lignes insérées dans la table 'job_skills'.")
except FileNotFoundError:
    print(f"Fichier non trouvé : {csv_file_path}")

# === Renommage des colonnes de manière uniforme ===
rename_queries = [
    "ALTER TABLE job_skills CHANGE `English name` english_name VARCHAR(255);",
    "ALTER TABLE job_skills CHANGE `Chinese name` chinese_name VARCHAR(255);",
    "ALTER TABLE job_skills CHANGE `Volume size` volume_size VARCHAR(50);",
    "ALTER TABLE job_skills CHANGE `Food intake` food_intake INT;",
    "ALTER TABLE job_skills CHANGE `night shift` night_shift BOOLEAN;",
    "ALTER TABLE job_skills CHANGE `Total skills` total_skills INT;",
    "ALTER TABLE job_skills CHANGE `Make a fire` make_a_fire BOOLEAN;",
    "ALTER TABLE job_skills CHANGE `watering` watering BOOLEAN;",
    "ALTER TABLE job_skills CHANGE `planting` planting BOOLEAN;",
    "ALTER TABLE job_skills CHANGE `generate electricity` generate_electricity BOOLEAN;",
    "ALTER TABLE job_skills CHANGE `manual` manual BOOLEAN;",
    "ALTER TABLE job_skills CHANGE `collection` collection BOOLEAN;",
    "ALTER TABLE job_skills CHANGE `logging` logging BOOLEAN;",
    "ALTER TABLE job_skills CHANGE `Mining` mining BOOLEAN;",
    "ALTER TABLE job_skills CHANGE `pharmaceutical` pharmaceutical BOOLEAN;",
    "ALTER TABLE job_skills CHANGE `cool down` cool_down BOOLEAN;",
    "ALTER TABLE job_skills CHANGE `pasture` pasture BOOLEAN;",
    "ALTER TABLE job_skills CHANGE `carry` carry BOOLEAN;",
    "ALTER TABLE job_skills CHANGE `Handling speed` handling_speed BOOLEAN;",
    "ALTER TABLE job_skills CHANGE `ranch items` ranch_items VARCHAR(255);",
    "ALTER TABLE job_skills CHANGE `pasture minimum output` pasture_minimum_output VARCHAR(255);",
    "ALTER TABLE job_skills CHANGE `The largest ranch` the_largest_ranch VARCHAR(255);"
]

print("\nRenommage des colonnes...")
for query in rename_queries:
    try:
        cursor.execute(query)
    except mariadb.Error as e:
        print(f"Erreur lors du renommage : {e}")

conn.commit()
print("Renommage terminé.")

# === Fermeture ===
cursor.close()
conn.close()
print("Connexion fermée.")
   
       


In [ ]:
# Création de la table palu_combat_attribute et insertion des données du csv

# === Configuration de la base de données ===
config = {
    'user': 'root',
    'password': 'root',
    'host': 'localhost',
    'port': 3307,
    'database': 'palworld_database'
}

# === Connexion à MariaDB ===
try:
    conn = mariadb.connect(**config)
    cursor = conn.cursor()
    print("Connexion réussie à MariaDB !")
except mariadb.Error as e:
    print(f"Erreur de connexion : {e}")
    exit(1)

# === Création de la table  ===
create_table_query = """
CREATE TABLE IF NOT EXISTS palu_combat_attribute (
    id INT AUTO_INCREMENT PRIMARY KEY,
    `Chinese name` VARCHAR(255),
    `Name` VARCHAR(255),
    `CodeName` VARCHAR(255),
    `OverrideNameTextID` VARCHAR(255),
    `NamePrefixID` VARCHAR(255),
    `OverridePartnerSkillTextID` VARCHAR(255),
    `IsPal` BOOLEAN,
    `Tribe` VARCHAR(255),
    `BPClass` VARCHAR(255),
    `variant` VARCHAR(255),
    `Volume size` VARCHAR(255),
    `rarity` INT,
    `Element 1` VARCHAR(255),
    `Element 2` VARCHAR(255),
    `GenusCategory` VARCHAR(255),
    `Organization` VARCHAR(255),
    `weapon` VARCHAR(255),
    `WeaponEquip` VARCHAR(255),
    `nocturnal` BOOLEAN,
    `total_4D` INT,
    `HP` INT,
    `melee attack` INT,
    `Remote attack` INT,
    `defense` INT,
    `support` INT,
    `Speed of work` INT,
    `Level1Min` INT,
    `Level1Max` INT,
    `Level20Min` INT,
    `Level20Max` INT,
    `Level50Min` INT,
    `Level50Max` INT,
    `AIRResponse` VARCHAR(255),
    `AISightResponse` INT,
    `endurance` INT,
    `slow walking speed` INT,
    `walking speed` INT,
    `running speed` INT,
    `Riding sprint speed` INT,
    `damage multiplier` FLOAT,
    `catch rate` INT,
    `Experience multiplier` FLOAT,
    `price` INT,
    `Must bring entry 1` VARCHAR(255),
    `Must bring entry 2` VARCHAR(255),
    `Numerical description` VARCHAR(255),
    `Skill description` TEXT,
    `lv1.1` INT,
    `lv2.1` INT,
    `lv3.1` INT,
    `lv4.1` INT,
    `lv5.1` INT
);
"""

try:
    cursor.execute(create_table_query)
    conn.commit()
    print("Table 'palu_combat_attribute' prête.")
except mariadb.Error as e:
    print(f"Erreur création table : {e}")
    conn.close()
    exit(1)


# === Lecture CSV avec Pandas ===
csv_file_path = r'C:/Users/ndiay/Desktop/lptf/projets/IA/DATA_ANALYSIS/JUPITER_NOTEBOOK/Analyse_pals/pals-analysis/data/data2.0/Clean_Data--Palu-combat-attribute.csv'

try:
    df = pd.read_csv(csv_file_path)
    print(f"\nLe fichier CSV contient {df.shape[0]} lignes et {df.shape[1]} colonnes.")
except FileNotFoundError:
    print(f"Fichier non trouvé : {csv_file_path}")
    conn.close()
    exit(1)


# === Nombre de colonnes dans la table ===
cursor.execute("SELECT COUNT(*) FROM information_schema.columns WHERE table_name = 'palu_combat_attribute' AND table_schema = 'palworld_database'")
column_count = cursor.fetchone()[0]
print(f"La table 'palu_combat_attribute' contient maintenant {column_count} colonnes.")

# === Fonction pour convertir les valeurs ===
def convert(value, col_name):
    if value == '' or str(value).lower() == 'none':
        return None
    if col_name.lower() in ['ispal', 'nocturnal']:
        return str(value).strip().lower() == 'true'
    value = value.replace(',', '.')
    try:
        if '.' in value:
            return float(value)
        return int(value)
    except ValueError:
        return value

# === Fonction de nettoyage pour les colonnes name ===
def clean_name(name):
    if name is None:
        return None
    # Convertir en minuscules, supprimer les underscores et les espaces
    return str(name).lower().replace('_', '').replace(' ', '')

# === Insertion des données ===
try:
    with open(csv_file_path, mode='r', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        headers = reader.fieldnames

        sql_headers = [h for h in headers]  # on conserve les noms initiaux
        placeholders = ', '.join(['%s'] * len(sql_headers))
        field_names = ', '.join(f'`{col}`' for col in sql_headers)
        insert_query = f"INSERT INTO palu_combat_attribute ({field_names}) VALUES ({placeholders})"

        count = 0
        for row in reader:
            values = []
            for i, h in enumerate(headers):
                value = convert(row[h], h)
                # Nettoyer spécifiquement les colonnes "Chinese name", "Name" et "CodeName"
                if h.lower() in ['chinese name', 'name', 'codename']:
                    value = clean_name(value)
                values.append(value)
            try:
                cursor.execute(insert_query, values)
                count += 1
            except mariadb.Error as e:
                print(f"Erreur insertion ligne {count + 1}: {e} — valeurs: {values}")

        conn.commit()
        print(f"\n{count} lignes insérées dans la table 'palu_combat_attribute'.")
except FileNotFoundError:
    print(f"Fichier non trouvé : {csv_file_path}")

# === Renommage des colonnes  ===
rename_queries = [
    "ALTER TABLE palu_combat_attribute CHANGE `Chinese name` chinese_name VARCHAR(255);",
    "ALTER TABLE palu_combat_attribute CHANGE `Name` name VARCHAR(255);",
    "ALTER TABLE palu_combat_attribute CHANGE `CodeName` code_name VARCHAR(255);",
    "ALTER TABLE palu_combat_attribute CHANGE `OverrideNameTextID` override_name_text_id VARCHAR(255);",
    "ALTER TABLE palu_combat_attribute CHANGE `NamePrefixID` name_prefix_id VARCHAR(255);",
    "ALTER TABLE palu_combat_attribute CHANGE `OverridePartnerSkillTextID` override_partner_skill_text_id VARCHAR(255);",
    "ALTER TABLE palu_combat_attribute CHANGE `IsPal` is_pal BOOLEAN;",
    "ALTER TABLE palu_combat_attribute CHANGE `Tribe` tribe VARCHAR(255);",
    "ALTER TABLE palu_combat_attribute CHANGE `BPClass` bp_class VARCHAR(255);",
    "ALTER TABLE palu_combat_attribute CHANGE `Volume size` volume_size VARCHAR(255);",
    "ALTER TABLE palu_combat_attribute CHANGE `Element 1` element_1 VARCHAR(255);",
    "ALTER TABLE palu_combat_attribute CHANGE `Element 2` element_2 VARCHAR(255);",
    "ALTER TABLE palu_combat_attribute CHANGE `GenusCategory` genus_category VARCHAR(255);",
    "ALTER TABLE palu_combat_attribute CHANGE `Organization` organization VARCHAR(255);",
    "ALTER TABLE palu_combat_attribute CHANGE `weapon` weapon VARCHAR(255);",
    "ALTER TABLE palu_combat_attribute CHANGE `WeaponEquip` weapon_equip VARCHAR(255);",
    "ALTER TABLE palu_combat_attribute CHANGE `nocturnal` nocturnal BOOLEAN;",
    "ALTER TABLE palu_combat_attribute CHANGE `total_4D` total_4d INT;",
    "ALTER TABLE palu_combat_attribute CHANGE `melee attack` melee_attack INT;",
    "ALTER TABLE palu_combat_attribute CHANGE `Remote attack` remote_attack INT;",
    "ALTER TABLE palu_combat_attribute CHANGE `Speed of work` speed_of_work INT;",
    "ALTER TABLE palu_combat_attribute CHANGE `Level1Min` level1_min INT;",
    "ALTER TABLE palu_combat_attribute CHANGE `Level1Max` level1_max INT;",
    "ALTER TABLE palu_combat_attribute CHANGE `Level20Min` level20_min INT;",
    "ALTER TABLE palu_combat_attribute CHANGE `Level20Max` level20_max INT;",
    "ALTER TABLE palu_combat_attribute CHANGE `Level50Min` level50_min INT;",
    "ALTER TABLE palu_combat_attribute CHANGE `Level50Max` level50_max INT;",
    "ALTER TABLE palu_combat_attribute CHANGE `AIRResponse` air_response VARCHAR(255);",
    "ALTER TABLE palu_combat_attribute CHANGE `AISightResponse` ai_sight_response INT;",
    "ALTER TABLE palu_combat_attribute CHANGE `slow walking speed` slow_walking_speed INT;",
    "ALTER TABLE palu_combat_attribute CHANGE `walking speed` walking_speed INT;",
    "ALTER TABLE palu_combat_attribute CHANGE `running speed` running_speed INT;",
    "ALTER TABLE palu_combat_attribute CHANGE `Riding sprint speed` riding_sprint_speed INT;",
    "ALTER TABLE palu_combat_attribute CHANGE `damage multiplier` damage_multiplier FLOAT;",
    "ALTER TABLE palu_combat_attribute CHANGE `catch rate` catch_rate INT;",
    "ALTER TABLE palu_combat_attribute CHANGE `Experience multiplier` experience_multiplier FLOAT;",
    "ALTER TABLE palu_combat_attribute CHANGE `Must bring entry 1` must_bring_entry_1 VARCHAR(255);",
    "ALTER TABLE palu_combat_attribute CHANGE `Must bring entry 2` must_bring_entry_2 VARCHAR(255);",
    "ALTER TABLE palu_combat_attribute CHANGE `Numerical description` numerical_description VARCHAR(255);",
    "ALTER TABLE palu_combat_attribute CHANGE `Skill description` skill_description TEXT;",
    "ALTER TABLE palu_combat_attribute CHANGE `lv1.1` lv1_1 INT;",
    "ALTER TABLE palu_combat_attribute CHANGE `lv2.1` lv2_1 INT;",
    "ALTER TABLE palu_combat_attribute CHANGE `lv3.1` lv3_1 INT;",
    "ALTER TABLE palu_combat_attribute CHANGE `lv4.1` lv4_1 INT;",
    "ALTER TABLE palu_combat_attribute CHANGE `lv5.1` lv5_1 INT;"
]

print("\nRenommage des colonnes...")
for query in rename_queries:
    try:
        cursor.execute(query)
    except mariadb.Error as e:
        print(f"Erreur lors du renommage : {e}")

conn.commit()
print("Renommage terminé.")

# === Fermeture connexion ===
cursor.close()
conn.close()
print("Connexion fermée.")
    



In [ ]:
# Création table refresh et insertion des données du csv
# === Configuration de la base de données ===
config = {
    'user': 'root',
    'password': 'root',
    'host': 'localhost',
    'port': 3307,
    'database': 'palworld_database'
}
# === Connexion à MariaDB ===
try:
    conn = mariadb.connect(**config)
    cursor = conn.cursor()
    print("Connexion réussie à MariaDB !")
except mariadb.Error as e:
    print(f"Erreur de connexion : {e}")
    exit(1)
# === Création de la table avec tous les champs ===
create_table_query = """
CREATE TABLE IF NOT EXISTS palu_refresh_level (
    id INT AUTO_INCREMENT PRIMARY KEY,
    `name` VARCHAR(255),
    `minimum_level` INT,
    `maximum_level` INT,
    `fecondity` INT,
    `pallu_refresh_type` VARCHAR(255),
    `night_only` BOOLEAN,
    `refresh_area` VARCHAR(255)
);
"""
try:
    cursor.execute(create_table_query)
    conn.commit()
    print("Table 'palu_refresh_level' prête.")
except mariadb.Error as e:
    print(f"Erreur création table : {e}")
    conn.close()
    exit(1)
# === Lecture rapide CSV avec pandas pour afficher nombre lignes/colonnes ===
csv_file_path = r'C:/Users/ndiay/Desktop/lptf/projets/IA/DATA_ANALYSIS/JUPITER_NOTEBOOK/Analyse_pals/pals-analysis/data/data2.0/Clean_Data-Palu-refresh-level.csv'
try:
    df = pd.read_csv(csv_file_path)
    print(f"\nLe fichier CSV contient {df.shape[0]} lignes et {df.shape[1]} colonnes.")
except FileNotFoundError:
    print(f"Fichier non trouvé : {csv_file_path}")
    conn.close()
    exit(1)
# === Fonction pour convertir les valeurs ===
def convert(value, field=None):
    if value == '' or str(value).lower() == 'none':
        return None
    value = str(value).replace(',', '.')  # gestion des virgules décimales
    if field == 'night_only':
        return 1 if str(value).lower() == 'true' else 0
    try:
        if '.' in value:
            return float(value)
        return int(value)
    except ValueError:
        return value  # retourner chaîne

# === Fonction de nettoyage pour la colonne name ===
def clean_name(name):
    if name is None:
        return None
    # Convertir en minuscules, supprimer les underscores et les espaces
    return str(name).lower().replace('_', '').replace(' ', '')

# === Lecture CSV & insertion classique avec csv.DictReader ===
try:
    with open(csv_file_path, mode='r', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        headers = reader.fieldnames
        # Transformation noms colonnes pour SQL
        sql_headers = [h.strip().replace(' ', '_').replace('.', '_').replace('-', '_') for h in headers]
        placeholders = ', '.join(['%s'] * len(headers))
        field_names = ', '.join(f'`{col}`' for col in sql_headers)
        insert_query = f"INSERT INTO palu_refresh_level ({field_names}) VALUES ({placeholders})"
        count = 0
        for row in reader:
            values = []
            for i, h in enumerate(headers):
                value = convert(row[h], h.replace(' ', '_'))
                # Nettoyer spécifiquement la colonne "name"
                if h.lower() == 'name' or sql_headers[i].lower() == 'name':
                    value = clean_name(value)
                values.append(value)
            try:
                cursor.execute(insert_query, values)
                count += 1
            except mariadb.Error as e:
                print(f"Erreur insertion ligne {count + 1} : {e} — valeurs: {values}")
        conn.commit()
        print(f"\n{count} lignes insérées dans la table 'palu_refresh_level'.")
except FileNotFoundError:
    print(f"Fichier non trouvé : {csv_file_path}")
# === Fermeture connexion ===
cursor.close()
conn.close()
print("Connexion fermée.")

In [ ]:
## Création table & Insertion des données CSV dans la table boss_comparison
# === Config base de données ===
config = {
    'user': 'root',
    'password': 'root',
    'host': 'localhost',
    'port': 3307,
    'database': 'palworld_database'
}
# === Connexion ===
try:
    conn = mariadb.connect(**config)
    cursor = conn.cursor()
    print("Connexion réussie à MariaDB !")
except mariadb.Error as e:
    print(f"Erreur de connexion : {e}")
    exit(1)
# === Création de la table adaptée ===
create_table_query = """
CREATE TABLE IF NOT EXISTS ordinary_boss_comparison (
    id INT AUTO_INCREMENT PRIMARY KEY,
    `name` VARCHAR(255),
    `HP` INT,
    `Remote_attack` INT,
    `Riding_speed_BOSS_is_100_higher` INT
);
"""
try:
    cursor.execute(create_table_query)
    conn.commit()
    print("Table 'ordinary_boss_comparison' prête.")
except mariadb.Error as e:
    print(f"Erreur création table : {e}")
    conn.close()
    exit(1)
# === Affichage nombre colonnes et lignes dans la table avant insertion ===
try:
    cursor.execute("SELECT COUNT(*) FROM ordinary_boss_comparison")
    count_rows = cursor.fetchone()[0]
    count_columns = len([column[0] for column in cursor.description])
    print(f"\nAvant insertion : {count_rows} lignes, {count_columns} colonnes dans la table 'ordinary_boss_comparison'.")
except mariadb.Error as e:
    print(f"Erreur comptage lignes table : {e}")
# === Lecture rapide CSV avec pandas pour afficher nombre lignes/colonnes ===
csv_file_path = r'C:/Users/ndiay/Desktop/lptf/projets/IA/DATA_ANALYSIS/JUPITER_NOTEBOOK/Analyse_pals/pals-analysis/data/data2.0/Clean_Data-comparison-of-ordinary-BOSS-attributes.csv'
try:
    df = pd.read_csv(csv_file_path)
    print(f"\nLe fichier CSV contient {df.shape[0]} lignes et {df.shape[1]} colonnes.")
except FileNotFoundError:
    print(f"Fichier non trouvé : {csv_file_path}")
    conn.close()
    exit(1)
# === Fonction de conversion adaptée ===
def convert(value):
    if value == '' or str(value).lower() == 'none':
        return None
    value = str(value).replace(',', '.')  # gestion décimales françaises
    try:
        if '.' in value:
            return float(value)
        return int(value)
    except ValueError:
        return value  # texte

# === Fonction de nettoyage pour la colonne name ===
def clean_name(name):
    if name is None:
        return None
    # Convertir en minuscules, supprimer les underscores et les espaces
    return str(name).lower().replace('_', '').replace(' ', '')

# === Lecture CSV & insertion classique ===
try:
    with open(csv_file_path, mode='r', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        headers = reader.fieldnames
        # Adaptation noms colonnes pour SQL 
        sql_headers = [h.replace(' ', '_').replace('(', '').replace(')', '') for h in headers]
        placeholders = ', '.join(['%s'] * len(headers))
        field_names = ', '.join(f'`{col}`' for col in sql_headers)
        insert_query = f"INSERT INTO ordinary_boss_comparison ({field_names}) VALUES ({placeholders})"
        count = 0
        for row in reader:
            values = []
            for i, h in enumerate(headers):
                value = convert(row[h])
                # Nettoyer  la colonne "name"
                if h.lower() == 'name' or sql_headers[i].lower() == 'name':
                    value = clean_name(value)
                values.append(value)
            try:
                cursor.execute(insert_query, values)
                count += 1
            except mariadb.Error as e:
                print(f"Erreur insertion ligne {count + 1}: {e} — valeurs: {values}")
        conn.commit()
        print(f"\n{count} lignes insérées dans la table 'ordinary_boss_comparison'.")
except FileNotFoundError:
    print(f"Fichier non trouvé : {csv_file_path}")
# === Fermeture connexion ===
cursor.close()
conn.close()
print("Connexion fermée.")

In [ ]:
# Création table tower_bosses & Insertion des données dans les tables

# === Importations nécessaires ===
import mariadb
import pandas as pd
import csv

# === Config base de données ===
config = {
    'user': 'root',
    'password': 'root',
    'host': 'localhost',
    'port': 3307,
    'database': 'palworld_database'
}

# === Connexion ===
try:
    conn = mariadb.connect(**config)
    cursor = conn.cursor()
    print("Connexion réussie à MariaDB !")
except mariadb.Error as e:
    print(f"Erreur de connexion : {e}")
    exit(1)

# === Création de la table tower_bosses ===
create_table_query = """
CREATE TABLE IF NOT EXISTS tower_bosses (
    id INT AUTO_INCREMENT PRIMARY KEY,
    `name` VARCHAR(255),
    `HP` INT,
    `melee attack` INT,
    `remote attack` INT,
    `defense` INT,
    `Support` INT,
    `experience ratio` INT,
    `slow walking speed` INT,
    `walking speed` INT,
    `running speed` INT,
    `riding speed` INT,
    `Handling speed` INT,
    `ignore the bluntness` BOOLEAN,
    `ignore displacement` BOOLEAN,
    `BiologicalGrade` INT,
    `endurance` INT,
    `fecundity` INT
);
"""

try:
    cursor.execute(create_table_query)
    conn.commit()
    print("Table 'tower_bosses' prête.")
except mariadb.Error as e:
    print(f"Erreur création table : {e}")
    conn.close()
    exit(1)

# === Lecture CSV avec Pandas ===
csv_file_path = r'C:/Users/ndiay/Desktop/lptf/projets/IA/DATA_ANALYSIS/JUPITER_NOTEBOOK/Analyse_pals/pals-analysis/data/data2.0/Clean_Data-Tower-BOSS-attribute-comparison.csv'

try:
    df = pd.read_csv(csv_file_path)
    print(f"\nLe fichier CSV contient {df.shape[0]} lignes et {df.shape[1]} colonnes.")
except FileNotFoundError:
    print(f"Fichier non trouvé : {csv_file_path}")
    conn.close()
    exit(1)

# === Nombre de colonnes dans la table tower_bosses ===
cursor.execute("SELECT COUNT(*) FROM information_schema.columns WHERE table_name = 'tower_bosses' AND table_schema = 'palworld_database'")
column_count = cursor.fetchone()[0]
print(f"La table 'tower_bosses' contient maintenant {column_count} colonnes.")

# === Fonction de conversion ===
def convert(value, col_name):
    if value == '' or str(value).lower() == 'none':
        return None
    if col_name.lower() in ['ignore the bluntness', 'ignore displacement']:
        return str(value).strip().lower() == 'true'
    value = str(value).replace(',', '.')
    try:
        if '.' in value:
            return float(value)
        return int(value)
    except ValueError:
        return value

# === Fonction de nettoyage pour la colonne name ===
def clean_name(name):
    if name is None:
        return None
    # Convertir en minuscules, supprimer les underscores et les espaces
    return str(name).lower().replace('_', '').replace(' ', '')

# === Insertion des données dans tower_bosses ===
try:
    with open(csv_file_path, mode='r', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        headers = reader.fieldnames

        # Encadrement des noms de colonnes avec des backticks pour gérer les espaces
        sql_headers = [f'`{h}`' for h in headers]
        placeholders = ', '.join(['%s'] * len(sql_headers))
        field_names = ', '.join(sql_headers)
        insert_query = f"INSERT INTO tower_bosses ({field_names}) VALUES ({placeholders})"

        count = 0
        for row in reader:
            values = []
            for i, h in enumerate(headers):
                value = convert(row[h], h)
                # Nettoyer spécifiquement la colonne "name"
                if h.lower() == 'name':
                    value = clean_name(value)
                values.append(value)
            try:
                cursor.execute(insert_query, values)
                count += 1
            except mariadb.Error as e:
                print(f"Erreur insertion ligne {count + 1}: {e} — valeurs: {values}")

        conn.commit()
        print(f"\n{count} lignes insérées dans la table 'tower_bosses'.")
except FileNotFoundError:
    print(f"Fichier non trouvé : {csv_file_path}")

# === Renommage des colonnes de manière uniforme ===
rename_queries = [
    "ALTER TABLE tower_bosses CHANGE `name` name VARCHAR(255);",
    "ALTER TABLE tower_bosses CHANGE `melee attack` melee_attack INT;",
    "ALTER TABLE tower_bosses CHANGE `remote attack` remote_attack INT;",
    "ALTER TABLE tower_bosses CHANGE `Support` support INT;",
    "ALTER TABLE tower_bosses CHANGE `experience ratio` experience_ratio INT;",
    "ALTER TABLE tower_bosses CHANGE `slow walking speed` slow_walking_speed INT;",
    "ALTER TABLE tower_bosses CHANGE `walking speed` walking_speed INT;",
    "ALTER TABLE tower_bosses CHANGE `running speed` running_speed INT;",
    "ALTER TABLE tower_bosses CHANGE `riding speed` riding_speed INT;",
    "ALTER TABLE tower_bosses CHANGE `Handling speed` handling_speed INT;",
    "ALTER TABLE tower_bosses CHANGE `ignore the bluntness` ignore_the_bluntness BOOLEAN;",
    "ALTER TABLE tower_bosses CHANGE `ignore displacement` ignore_displacement BOOLEAN;",
    "ALTER TABLE tower_bosses CHANGE `BiologicalGrade` biological_grade INT;"
]

print("\nRenommage des colonnes...")
for query in rename_queries:
    try:
        cursor.execute(query)
    except mariadb.Error as e:
        print(f"Erreur lors du renommage : {e}")

conn.commit()
print("Renommage terminé.")

# === Fermeture ===
cursor.close()
conn.close()
print("Connexion fermée.")

In [ ]:
#Création et remplissage de la table monsters
# === Config base de données ===
config = {
    'user': 'root',
    'password': 'root',
    'host': 'localhost',
    'port': 3307,
    'database': 'palworld_database'
}

# === Connexion ===
try:
    conn = mariadb.connect(**config)
    cursor = conn.cursor()
    print("Connexion réussie à MariaDB !")
except mariadb.Error as e:
    print(f"Erreur de connexion : {e}")
    exit(1)

# === Création de la table ===
create_table_query = """
CREATE TABLE IF NOT EXISTS monsters (
    id INT AUTO_INCREMENT PRIMARY KEY,
    `OverrideNameTextID` VARCHAR(255),
    `OverridePartnerSkillTextID` VARCHAR(255),
    `IsPal` INT,
    `Tribe` VARCHAR(255),
    `BPClass` VARCHAR(255),
    `Pictorial ID` INT,
    `ZukanIndexSuffix` FLOAT,
    `Size` VARCHAR(50),
    `rarity` INT,
    `Element 1` VARCHAR(50),
    `Element 2` VARCHAR(50),
    `GenusCategory` VARCHAR(50),
    `Organization` FLOAT,
    `weapon` FLOAT,
    `WeaponEquip` INT,
    `HP` INT,
    `melee attack` INT,
    `Remote attack` INT,
    `defense` INT,
    `support` INT,
    `CraftSpeed` INT,
    `damageMultiplier` VARCHAR(50),
    `CaptureProbability` VARCHAR(50),
    `ExperienceMultiplier` INT,
    `price` INT,
    `AIRResponse` VARCHAR(50),
    `AISightResponse` FLOAT,
    `slow walking speed` INT,
    `walking speed` INT,
    `running speed` INT,
    `Riding sprint speed` INT,
    `Handling speed` INT,
    `IsBoss` INT,
    `IsTowerBoss` INT,
    `BattleBGM` VARCHAR(255),
    `IgnoreLeanBack` INT,
    `IgnoreBlowAway` INT,
    `MaxFullStomach` INT,
    `FullStomachDecreaseRate` INT,
    `FoodAmount` INT,
    `ViewingDistance` INT,
    `ViewingAngle` INT,
    `HearingRate` INT,
    `NooseTrap` INT,
    `Nocturnal` INT,
    `BiologicalGrade` INT,
    `Predator` INT,
    `Edible` INT,
    `endurance` INT,
    `Male probability` INT,
    `fecundity` INT,
    `Breathing fire` INT,
    `watering` INT,
    `planting` INT,
    `generate electricity` INT,
    `manual` INT,
    `collection` INT,
    `logging` INT,
    `Mining` INT,
    `pharmaceutical` INT,
    `cool down` INT,
    `carry` INT,
    `pasture` INT,
    `Passive skill 1` VARCHAR(255),
    `Passive skill 2` VARCHAR(255),
    `Passive skill 3` FLOAT,
    `Passive skill 4` FLOAT
);
"""

try:
    cursor.execute(create_table_query)
    conn.commit()
    print("Table 'monsters' prête.")
except mariadb.Error as e:
    print(f"Erreur création table : {e}")
    conn.close()
    exit(1)

# === Fonction de conversion des valeurs ===
def convert(value):
    if value == '' or value == 'None':
        return None
    try:
        value = value.replace(',', '.')
        if '.' in value:
            return float(value)
        return int(value)
    except ValueError:
        return value

# === Fonction de nettoyage pour la colonne OverrideNameTextID ===
def clean_override_name(name):
    if name is None:
        return None
    # Convertir en minuscules, supprimer les underscores et les espaces
    return str(name).lower().replace('_', '').replace(' ', '')

# === Chemin vers le fichier CSV ===
csv_file_path = r'C:/Users/ndiay/Desktop/lptf/projets/IA/DATA_ANALYSIS/JUPITER_NOTEBOOK/Analyse_pals/pals-analysis/data/data2.0/Clean_Data-hide-pallu-attributes.csv'

# === Lecture du CSV et insertion ===
try:
    with open(csv_file_path, mode='r', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        headers = reader.fieldnames

        placeholders = ', '.join(['%s'] * len(headers))
        field_names = ', '.join(f'`{h}`' for h in headers)
        insert_query = f"INSERT INTO monsters ({field_names}) VALUES ({placeholders})"

        count = 0
        for row in reader:
            values = []
            for h in headers:
                value = convert(row[h])
                # Nettoyer spécifiquement la colonne "OverrideNameTextID"
                if h == 'OverrideNameTextID':
                    value = clean_override_name(value)
                values.append(value)
            try:
                cursor.execute(insert_query, values)
                count += 1
            except mariadb.Error as e:
                print(f"Erreur insertion ligne {count + 1}: {e} — valeurs: {values}")

        conn.commit()
        print(f"{count} lignes insérées dans la table 'monsters'.")
except FileNotFoundError:
    print(f"Fichier non trouvé : {csv_file_path}")

# === Renommage des colonnes ===
print("Renommage des colonnes en cours...")

rename_queries = [
    "ALTER TABLE monsters CHANGE `Pictorial ID` `Pictorial_ID` INT",
    "ALTER TABLE monsters CHANGE `Element 1` `Element_1` VARCHAR(50)",
    "ALTER TABLE monsters CHANGE `Element 2` `Element_2` VARCHAR(50)",
    "ALTER TABLE monsters CHANGE `melee attack` `melee_attack` INT",
    "ALTER TABLE monsters CHANGE `Remote attack` `Remote_attack` INT",
    "ALTER TABLE monsters CHANGE `slow walking speed` `slow_walking_speed` INT",
    "ALTER TABLE monsters CHANGE `walking speed` `walking_speed` INT",
    "ALTER TABLE monsters CHANGE `running speed` `running_speed` INT",
    "ALTER TABLE monsters CHANGE `Riding sprint speed` `Riding_sprint_speed` INT",
    "ALTER TABLE monsters CHANGE `Handling speed` `Handling_speed` INT",
    "ALTER TABLE monsters CHANGE `Male probability` `Male_probability` INT",
    "ALTER TABLE monsters CHANGE `Breathing fire` `Breathing_fire` INT",
    "ALTER TABLE monsters CHANGE `generate electricity` `generate_electricity` INT",
    "ALTER TABLE monsters CHANGE `cool down` `cool_down` INT",
    "ALTER TABLE monsters CHANGE `Passive skill 1` `Passive_skill_1` VARCHAR(255)",
    "ALTER TABLE monsters CHANGE `Passive skill 2` `Passive_skill_2` VARCHAR(255)",
    "ALTER TABLE monsters CHANGE `Passive skill 3` `Passive_skill_3` FLOAT",
    "ALTER TABLE monsters CHANGE `Passive skill 4` `Passive_skill_4` FLOAT"
]

try:
    for query in rename_queries:
        cursor.execute(query)

    conn.commit()
    print("Toutes les colonnes ont été renommées avec succès !")

except mariadb.Error as e:
    print(f"Erreur lors du renommage : {e}")

# === Comptage colonnes et lignes CSV ET table ===
try:
    with open(csv_file_path, mode='r', encoding='utf-8') as f:
        reader = csv.reader(f)
        csv_rows = list(reader)
        csv_col_count = len(csv_rows[0]) if csv_rows else 0
        csv_row_count = len(csv_rows) - 1 if len(csv_rows) > 0 else 0

    print(f"\nCSV : {csv_row_count} lignes, {csv_col_count} colonnes.")

    cursor.execute("SHOW COLUMNS FROM monsters")
    db_columns = cursor.fetchall()
    db_col_count = len(db_columns)

    cursor.execute("SELECT COUNT(*) FROM monsters")
    db_row_count = cursor.fetchone()[0]

    print(f"Table 'monsters' : {db_row_count} lignes, {db_col_count} colonnes.")

except Exception as e:
    print(f"Erreur lors du comptage des lignes/colonnes : {e}")

# === Fermeture ===
cursor.close()
conn.close()
print("\nConnexion fermée.")

In [ ]:
# === Affichage de la structure et des données de job_skills ===

# Reconnexion à la base de données
try:
    conn = mariadb.connect(**config)
    cursor = conn.cursor()
    print("Connexion réussie à MariaDB !")
except mariadb.Error as e:
    print(f"Erreur de connexion : {e}")
    exit(1)

# 1. Affichage de la structure de la table
print("\n=== Structure de la table 'job_skills' ===")
try:
    cursor.execute("DESCRIBE job_skills")
    structure = cursor.fetchall()
    for column in structure:
        print(f"Colonne: {column[0]:<25} Type: {column[1]}")
except mariadb.Error as e:
    print(f"Erreur récupération structure : {e}")

# 2. Affichage des 3 premières lignes
print("\n=== 3 premières lignes de la table ===")
try:
    cursor.execute("SELECT * FROM job_skills LIMIT 3")
    rows = cursor.fetchall()
    
    # Récupération des noms de colonnes
    cursor.execute("SELECT column_name FROM information_schema.columns WHERE table_name = 'job_skills' AND table_schema = 'palworld_database' ORDER BY ordinal_position")
    columns = [col[0] for col in cursor.fetchall()]
    
    # Affichage pandas
    import pandas as pd
    df = pd.DataFrame(rows, columns=columns)
    print(df)
except mariadb.Error as e:
    print(f"Erreur récupération données : {e}")

# Fermeture de la connexion
cursor.close()
conn.close()
print("\nConnexion fermée.")

In [ ]:
# === Affichage de la structure et des données de la table palu_combat_attribute ===

# Reconnexion à la base de données
try:
    conn = mariadb.connect(**config)
    cursor = conn.cursor()
    print("Connexion réussie à MariaDB !")
except mariadb.Error as e:
    print(f"Erreur de connexion : {e}")
    exit(1)

# 1. Affichage de la structure de la table
print("\n=== Structure de la table 'palu_combat_attribute' ===")
try:
    cursor.execute("DESCRIBE palu_combat_attribute")
    structure = cursor.fetchall()
    for column in structure:
        print(f"Colonne: {column[0]:<25} Type: {column[1]}")
except mariadb.Error as e:
    print(f"Erreur récupération structure : {e}")

# 2. Affichage des 3 premières lignes
print("\n=== 3 premières lignes de la table ===")
try:
    cursor.execute("SELECT * FROM palu_combat_attribute LIMIT 3")
    rows = cursor.fetchall()
    
    # Récupération des noms de colonnes
    cursor.execute("SELECT column_name FROM information_schema.columns WHERE table_name = 'palu_combat_attribute' AND table_schema = 'palworld_database' ORDER BY ordinal_position")
    columns = [col[0] for col in cursor.fetchall()]
    
    # Affichage avec pandas 
    import pandas as pd
    df = pd.DataFrame(rows, columns=columns)
    print(df)
except mariadb.Error as e:
    print(f"Erreur récupération données : {e}")

# Fermeture de la connexion
cursor.close()
conn.close()
print("\nConnexion fermée.")

In [ ]:
# === Affichage de la structure et des données de palu_refresh_level ===

# Reconnexion à la base de données
try:
    conn = mariadb.connect(**config)
    cursor = conn.cursor()
    print("Connexion réussie à MariaDB !")
except mariadb.Error as e:
    print(f"Erreur de connexion : {e}")
    exit(1)

# 1. Affichage de la structure de la table
print("\n=== Structure de la table 'palu_refresh_level' ===")
try:
    cursor.execute("DESCRIBE palu_refresh_level")
    structure = cursor.fetchall()
    for column in structure:
        print(f"Colonne: {column[0]:<20} Type: {column[1]}")
except mariadb.Error as e:
    print(f"Erreur récupération structure : {e}")

# 2. Affichage des 3 premières lignes
print("\n=== 3 premières lignes de la table ===")
try:
    cursor.execute("SELECT * FROM palu_refresh_level LIMIT 3")
    rows = cursor.fetchall()
    
    # Récupération des noms de colonnes
    cursor.execute("SELECT column_name FROM information_schema.columns WHERE table_name = 'palu_refresh_level' AND table_schema = 'palworld_database' ORDER BY ordinal_position")
    columns = [col[0] for col in cursor.fetchall()]
    
    # Affichage avec pandas pour une meilleure lisibilité
    import pandas as pd
    df = pd.DataFrame(rows, columns=columns)
    print(df)
except mariadb.Error as e:
    print(f"Erreur récupération données : {e}")

# 3. Affichage du nombre total d'entrées
try:
    cursor.execute("SELECT COUNT(*) FROM palu_refresh_level")
    count = cursor.fetchone()[0]
    print(f"\nLa table contient actuellement {count} entrées.")
except mariadb.Error as e:
    print(f"Erreur comptage lignes : {e}")

# Fermeture de la connexion
cursor.close()
conn.close()
print("\nConnexion fermée.")

In [ ]:
# Structure de la table ordinary_boss_comparison et controle des 3 premières lignes

# Configuration pour se connecter à la base de données
config = {
    'user': 'root',
    'password': 'root',
    'host': 'localhost',
    'port': 3307,
    'database': 'palworld_database'
}

try:
    # Connexion à la base de données
    conn = mariadb.connect(**config)
    cursor = conn.cursor()

    # Afficher la structure de la table
    cursor.execute("DESCRIBE ordinary_boss_comparison")
    structure = cursor.fetchall()
    print("Structure de la table 'ordinary_boss_comparison' :")
    for column in structure:
        print(f"Colonne: {column[0]}, Type: {column[1]}")

    # Afficher les trois premières lignes de la table
    cursor.execute("SELECT * FROM ordinary_boss_comparison LIMIT 3")
    rows = cursor.fetchall()
    print("\nTrois premières lignes de la table :")
    for row in rows:
        print(row)

except mariadb.Error as e:
    print(f"Erreur lors de la connexion à MariaDB : {e}")

finally:
    # Fermer la connexion
    if 'conn' in locals():
        cursor.close()
        conn.close()
        print("\nConnexion à la base de données fermée.")


In [ ]:
# === Affichage de la structure et des données de tower_bosses ===

# Reconnexion à la base de données
try:
    conn = mariadb.connect(**config)
    cursor = conn.cursor()
    print("Connexion réussie à MariaDB !")
except mariadb.Error as e:
    print(f"Erreur de connexion : {e}")
    exit(1)

# 1. Affichage de la structure de la table
print("\n=== Structure de la table 'tower_bosses' ===")
try:
    cursor.execute("DESCRIBE tower_bosses")
    structure = cursor.fetchall()
    for column in structure:
        print(f"Colonne: {column[0]:<20} Type: {column[1]}")
except mariadb.Error as e:
    print(f"Erreur récupération structure : {e}")

# 2. Affichage des 3 premières lignes
print("\n=== 3 premières lignes de la table ===")
try:
    cursor.execute("SELECT * FROM tower_bosses LIMIT 3")
    rows = cursor.fetchall()
    
    # Récupération des noms de colonnes
    cursor.execute("SELECT column_name FROM information_schema.columns WHERE table_name = 'tower_bosses' AND table_schema = 'palworld_database' ORDER BY ordinal_position")
    columns = [col[0] for col in cursor.fetchall()]
    
    # Affichage avec pandas pour une meilleure lisibilité
    import pandas as pd
    df = pd.DataFrame(rows, columns=columns)
    print(df)
except mariadb.Error as e:
    print(f"Erreur récupération données : {e}")

# 3. Affichage du nombre total d'entrées
try:
    cursor.execute("SELECT COUNT(*) FROM tower_bosses")
    count = cursor.fetchone()[0]
    print(f"\nLa table contient actuellement {count} entrées.")
except mariadb.Error as e:
    print(f"Erreur comptage lignes : {e}")

# Fermeture de la connexion
cursor.close()
conn.close()
print("\nConnexion fermée.")

In [7]:
# === Affichage de la structure et des données de monsters ===
# === Config base de données ===
config = {
    'user': 'root',
    'password': 'root',
    'host': 'localhost',
    'port': 3307,
    'database': 'palworld_database'
}

# Reconnexion à la base de données
try:
    conn = mariadb.connect(**config)
    cursor = conn.cursor()
    print("Connexion réussie à MariaDB !")
except mariadb.Error as e:
    print(f"Erreur de connexion : {e}")
    exit(1)

# 1. Affichage de la structure de la table
print("\n=== Structure de la table 'monsters' ===")
try:
    cursor.execute("DESCRIBE monsters")
    structure = cursor.fetchall()
    for column in structure:
        print(f"Colonne: {column[0]:<25} Type: {column[1]}")
except mariadb.Error as e:
    print(f"Erreur récupération structure : {e}")

# 2. Affichage des 3 premières lignes
print("\n=== 3 premières lignes de la table ===")
try:
    cursor.execute("SELECT * FROM monsters LIMIT 3")
    rows = cursor.fetchall()
    
    cursor.execute("SELECT column_name FROM information_schema.columns WHERE table_name = 'monsters' AND table_schema = 'palworld_database' ORDER BY ordinal_position")
    columns = [col[0] for col in cursor.fetchall()]
    
    import pandas as pd
    df = pd.DataFrame(rows, columns=columns)
    print(df)
except mariadb.Error as e:
    print(f"Erreur récupération données : {e}")

# 3. Affichage du nombre total d'entrées
try:
    cursor.execute("SELECT COUNT(*) FROM monsters")
    count = cursor.fetchone()[0]
    print(f"\nLa table contient actuellement {count} entrées.")
except mariadb.Error as e:
    print(f"Erreur comptage lignes : {e}")

# Fermeture de la connexion
cursor.close()
conn.close()
print("\nConnexion fermée.")

Connexion réussie à MariaDB !

=== Structure de la table 'monsters' ===
Colonne: id                        Type: int(11)
Colonne: OverrideNameTextID        Type: varchar(255)
Colonne: OverridePartnerSkillTextID Type: varchar(255)
Colonne: IsPal                     Type: int(11)
Colonne: Tribe                     Type: varchar(255)
Colonne: BPClass                   Type: varchar(255)
Colonne: Pictorial_ID              Type: int(11)
Colonne: ZukanIndexSuffix          Type: float
Colonne: Size                      Type: varchar(50)
Colonne: rarity                    Type: int(11)
Colonne: Element_1                 Type: varchar(50)
Colonne: Element_2                 Type: varchar(50)
Colonne: GenusCategory             Type: varchar(50)
Colonne: Organization              Type: float
Colonne: weapon                    Type: float
Colonne: WeaponEquip               Type: int(11)
Colonne: HP                        Type: int(11)
Colonne: melee_attack              Type: int(11)
Colonne: Remote